In [19]:
from dotenv import load_dotenv
from traveltimepy import time_filter

In [10]:
load_dotenv()

True

In [5]:
ttpy_locations = [{'id': 'Quäk4', "coords": {"lng": 13.30804, "lat": 52.57311}},
                  {'id': 'Rudi29', "coords": {"lng": 13.39095, "lat": 52.50675}},
                  {'id': 'Otis', "coords": {"lat": 52.57156, "lng": 13.30209}}]

Generate two searches with identical departure time, identical destination but two different origins

In [49]:
ttpy_dep_searches = [
    {
        "id": "testsearch_dep_Quäk4",
        "departure_location_id": "Quäk4",
        "arrival_location_ids": [
            "Rudi29",
        ],
        "departure_time": "2022-11-03T21:45:00+01:00",
        "travel_time": 3600,
        "properties": [
            "travel_time", "route", "distance"
        ],
        "transportation": {
            "type": "public_transport",
        },
    },
    {
        "id": "testsearch_dep_Otis",
        "departure_location_id": "Otis",
        "arrival_location_ids": [
            "Rudi29",
        ],
        "departure_time": "2022-11-03T21:45:00+01:00",
        "travel_time": 3600,
        "properties": [
            "travel_time", "route", "distance"
        ],
        "transportation": {
            "type": "public_transport",
        },
    }
]

In [50]:
r = time_filter(locations=ttpy_locations, departure_searches=ttpy_dep_searches)

Filter the results for route parts using public transport for better comparability:

In [51]:
routes = list(route["route"]["parts"] for result in r["results"] for location in result["locations"] for route in
              location["properties"])

In [60]:
public_transport_parts = [
    [{key: value for key, value in p.items() if key != "coords"} for p in route if p["type"] == "public_transport"] for
    route in routes]

One search returns a route that first uses the line `U6` from `U Otisstr. (Berlin)` in the opposite direction back to `U Holzhauser Str. (Berlin)` where it boards another train of the `U6`that arrives at `U Kochstr./Checkpoint Charlie (Berlin)` at 22:17:

In [64]:
public_transport_parts[1]

[{'id': 4,
  'type': 'public_transport',
  'mode': 'rail_underground',
  'directions': "Take a train (U6 (of Berliner Verkehrsbetriebe) line) from 'U Otisstr. (Berlin)' (leaves at 21:53) to 'U Holzhauser Str. (Berlin)' (arrives at 21:54) (1 stops)",
  'distance': 0,
  'travel_time': 60,
  'line': 'U6 (of Berliner Verkehrsbetriebe)',
  'departure_station': 'U Otisstr. (Berlin)',
  'arrival_station': 'U Holzhauser Str. (Berlin)',
  'departs_at': '21:53',
  'arrives_at': '21:54',
  'num_stops': 1},
 {'id': 6,
  'type': 'public_transport',
  'mode': 'rail_underground',
  'directions': "Take a train (U6 (of Berliner Verkehrsbetriebe) line) from 'U Holzhauser Str. (Berlin)' (leaves at 21:56) to 'U Kochstr./Checkpoint Charlie (Berlin)' (arrives at 22:17) (16 stops)",
  'distance': 0,
  'travel_time': 1337,
  'line': 'U6 (of Berliner Verkehrsbetriebe)',
  'departure_station': 'U Holzhauser Str. (Berlin)',
  'arrival_station': 'U Kochstr./Checkpoint Charlie (Berlin)',
  'departs_at': '21:56',
 

The other search returns a route that directly boards a `U6` at `U Otisstr. (Berlin)` going to `U Kochstr./Checkpoint Charlie (Berlin)` where it also arrives at 22:17. That means, that for no reason the first connection leaves the origin early than necessary in order to first take a train going the wrong direction for one stop, deboard, and take the next train in the correct directio while it should have obviously been possible to leave the origin 4 minutes later without arriving later at the destination by boarding the right train immediately that also the other search is using. This looks like some unexpected behavior of the routing algorithm for departure searches to me.

In [65]:
public_transport_parts[0]

[{'id': 6,
  'type': 'public_transport',
  'mode': 'rail_underground',
  'directions': "Take a train (U6 (of Berliner Verkehrsbetriebe) line) from 'U Otisstr. (Berlin)' (leaves at 21:57) to 'U Kochstr./Checkpoint Charlie (Berlin)' (arrives at 22:17) (15 stops)",
  'distance': 0,
  'travel_time': 1200,
  'line': 'U6 (of Berliner Verkehrsbetriebe)',
  'departure_station': 'U Otisstr. (Berlin)',
  'arrival_station': 'U Kochstr./Checkpoint Charlie (Berlin)',
  'departs_at': '21:57',
  'arrives_at': '22:17',
  'num_stops': 15}]